In [1]:
import json
import os
from library_dicom.dicom_processor.model.Series import Series 
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.model.SeriesCT import SeriesCT
from library_dicom.dicom_processor.model.csv_reader.MaskBuilder import MaskBuilder

from library_dicom.dicom_processor.tools.folders import *

import numpy as np
import SimpleITK as sitk

In [2]:
#LIST FROM PARSE_FILTERED_DICOM_DATASET

In [2]:
nifti_directory = '/media/salim/DD 2To/AHL2011_NIFTI'

csv_directory = '/media/salim/DD 2To/AHL2011_CSV_DATA/AHL2011-CSV'

In [4]:
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_list_path_csv.json'

In [8]:
data = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        for path in reader[info] :
            data.append(path)

In [9]:
dataset = []
for i in range(0,len(data),5):
    subliste = []
    subliste.append(data[i])
    subliste.append(data[i+1])
    subliste.append(data[i+2])
    subliste.append(data[i+3])
    subliste.append(data[i+4])
    dataset.append(subliste)

In [10]:
print(len(dataset))


617


In [ ]:
#GENERE LA LISTE DES JSON AVEC UN UNCONSTANT SPACING, SAVE IT AS JSON 

In [ ]:

from library_dicom.dicom_processor.tools.folders import *
liste_unconstant_z_spacing = []
for liste in dataset : 
    print(dataset.index(liste))
    for i in range(0,4,2) : 
        serie_objet = Series(liste[i])
        serie_objet.get_numpy_array()
        if serie_objet.get_z_spacing() == 'Unconstant Spacing' : 
            print(liste)
            liste_unconstant_z_spacing.append(liste)
            list_z_spacing = serie_objet.calculate_z_spacing()
            print(list_z_spacing)
            #liste_unconstant_z_spacing.append(list_z_spacing)


#save it as json

write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_problem_unconstant_spacing', liste_unconstant_z_spacing)

In [ ]:
from library_dicom.dicom_processor.tools.folders import *
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_problem_unconstant_spacing', liste_unconstant_z_spacing)

In [ ]:
#On enleve les series avec un unconstant spacing 
for serie in liste_unconstant_z_spacing : 
    dataset.remove(serie)

In [ ]:
#SAVE AS JSON THE FINAL LIST WITHOUT UNCONSTANT SPACING ERROR

In [ ]:
from library_dicom.dicom_processor.tools.folders import *
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_final_list_dataset', dataset)


In [ ]:
print(len(dataset)) #589


In [11]:
#RECHARGEZ LE JSON AVEC LES SERIES 
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_final_list_dataset.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

print(len(dataset))

589


In [ ]:
#IF MASK IS CORRECT => GENERATE NIFTI 
#IF PHILIPS, OR MASK FALSE => PUT SERIE IN A NEW LISTE 

In [ ]:
untreated_liste = []
#CT et PT pas dans le même ordre suivant les séries 
#voir pour les ranger avant PT en premier puis CT 


#error = []

#trier les philips des autres series fausses ici 


for serie in dataset : 
    try : 
        print(serie)
        print(dataset.index(serie))
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            manufacturer = serie_pt_objet.get_series_details()['series']['Manufacturer'] 
            if 'philips' in manufacturer.lower() :
                untreated_liste.append(serie)
            else : 
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
                size_matrix = serie_pt_objet.get_size_matrix()
                mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
                mask_4D = mask_objet.mask_array
                if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
                    print("MASK CORRECT")
                    #generation nifti PT
                    filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                    print("EXPORT NIFTI PT")
                    #generation nifti CT
                    serie_ct_objet = SeriesCT(serie[2])
                    serie_ct_objet.get_numpy_array()
                    filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                    serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                    print("EXPORT NIFTI CT")
                    #generation nifti mask
                    filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                    print("EXPORT NIFTI MASK")
                else : 
                    untreated_liste.append(serie)
        else : 
            serie_pt_objet = SeriesPT(serie[2])
            manufacturer = serie_pt_objet.get_series_details()['series']['Manufacturer'] 
            if 'philips' in manufacturer.lower() :
                untreated_liste.append(serie)
            else : 
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
                size_matrix = serie_pt_objet.get_size_matrix()
                mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
                mask_4D = mask_objet.mask_array
                if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
                    print("MASK CORRECT")
                    #generation nifti PT
                    filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                    print("EXPORT NIFTI PT")
                     #generation nifti CT
                    serie_ct_objet = SeriesCT(serie[0])
                    serie_ct_objet.get_numpy_array()
                    filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                    serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                    print("EXPORT NIFTI CT")
                    #generation nifti mask
                    filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                    print("EXPORT NIFTI MASK")
                else : 
                    untreated_liste.append(serie)
                   


    except Exception as err : 
        print(err)
        #error.append(serie)

        


In [ ]:
len(untreated_liste) #392 

In [ ]:
#SAVE THE SERIE WITH FALSE MASK 
from library_dicom.dicom_processor.tools.folders import *
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_serie_with_false_mask', untreated_liste)
print(len(untreated_liste))

In [ ]:
#SEARCH SERIE WITH ERROR ET SAVE IT AS JSON 

In [12]:
#IMPORTER JSON OF FINAL LISTE WITHOUT UNCONSTANT SPACING PROBLEM 
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_final_list_dataset.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

print(len(dataset))


589


In [16]:
#sauver les séries qui font des erreurs dans un autre json 
serie_with_error = []
for serie in dataset : 
    try : 
        #print(serie)
        print(dataset.index(serie))
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array

        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array

    except Exception as err : 
        print(serie)
        print(err)
        serie_with_error.append(serie)

from library_dicom.dicom_processor.tools.folders import *
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_serie_with_error', serie_with_error)
#On traitera après 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101021022/1.2.250.1.59.470.940100027.1.20130802113038.1925.124326/1.3.46.670589.28.2.10.4.9167.27921.2.460.0.1376045065/1.3.46.670589.28.2.10.4.9167.27921.2.460.0.1376045065', 'PT', '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101021022/1.2.250.1.59.470.940100027.1.20130802113038.1925.124326/1.2.840.113704.1.111.2068.1376042666.8/1.2.840.113704.1.111.2068.1376042666.8', 'CT', 'mus_ar_aug 9_2013_abr.csv']
float division by zero
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
1

In [18]:
len(serie_with_error)
serie_with_error.append(dataset[220])
serie_with_error.append(dataset[231])
serie_with_error.append(dataset[325])
serie_with_error.append(dataset[474])
serie_with_error.append(dataset[453])
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_serie_with_error', serie_with_error)
# => 22 séries avec des erreurs, à voir 

In [ ]:
220
231
325
474

Error generating result array time

 data '' does not match format '%Y%m%d%H%M%S'

453
Error generating result array unsupported operand type(s) for *: 'NoneType' and 'int'

#float division par 0
#index 255 is out of bounds for axis 0 with size 255

In [ ]:
#IMPORT SERIES WITH FALSE MASK,
#CHECK FLIP AXE Z + SMALL ERROR ON ELLIPSE 

In [13]:
#importer la liste untreated_serie pour la suite : 
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_serie_with_false_mask.json'
untreated_liste = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        untreated_liste.append(info)

print(len(untreated_liste)) #392

392


In [14]:
#SEPARE LES SERIES PHILIPS DES AUTRES SERIES UNTREATED
philips_serie = []
untreated_liste_2 = []
for serie in untreated_liste : 
    #print(untreated_liste.index(serie))
    try : 
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0]) 
            manufacturer = serie_pt_objet.get_series_details()['series']['Manufacturer'] 
            #si philips on retraitera plus tard 
            if 'philips' in manufacturer.lower() :
                philips_serie.append(serie)
            else : untreated_liste_2.append(serie)

        else : 
            serie_pt_objet = SeriesPT(serie[2]) 
            manufacturer = serie_pt_objet.get_series_details()['series']['Manufacturer'] 
            #si philips on retraitera plus tard 
            if 'philips' in manufacturer.lower() :
                philips_serie.append(serie)
            else : untreated_liste_2.append(serie)

    except Exception as err : 
        print(serie)
        print(err)


print(len(philips_serie))
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_untreated_philips_serie', philips_serie)
print(len(untreated_liste_2))
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_untreated_serie', untreated_liste_2)

203
189


In [3]:
#IMPORT JSON (serie with false mask, except Philips)
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_untreated_serie.json'
untreated_liste_2 = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        untreated_liste_2.append(info)

print(len(untreated_liste_2)) #189

189


In [4]:
#SUR LES LISTES NON PHILIPS, ON REGARDE
#SI ERREUR SUR SUV MAX MEAN SD SEULEMENT SUR DES ELLIPSES, SI OUI => GENERER NIFTI 

erreur_polygone_max = []
erreur_polygone_mean_sd = []
for serie in untreated_liste_2 : 
    print(untreated_liste_2.index(serie))
    try : 
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array

            if 'POLYGON' in mask_objet.ecart_suv_max(nifti_array) :
                print("PROBLEME SUV MAX POLYGON")
                erreur_polygone_max.append(serie)
            
            else : 

                result_mean = mask_objet.ecart_suv_mean(nifti_array)
                result_sd = mask_objet.ecart_SD(nifti_array)
                if (('POLYGON' in result_mean) and ('POLYGON' in result_sd)) : 
                    print("PROBLEME SUV MEAN ET SD POLYGONE")
                    print("ecart mean", result_mean)
                    print("ecart sd", result_sd)
                    erreur_polygone_mean_sd.append(serie)

                else : 
                    #si erreur sur les sd et mean QUE sur les ellipses, on tolère 
                    print("SMALL ERRORS ON ELLIPSE, MASK CORRECT")
                    #generation nifti PT
                    filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                    print("EXPORT NIFTI PT")
                    #generation nifti CT
                    serie_ct_objet = SeriesCT(serie[2])
                    serie_ct_objet.get_numpy_array()
                    filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                    serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                    print("EXPORT NIFTI CT")
                    #generation nifti mask
                    filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                    print("EXPORT NIFTI MASK")


        else : 
            serie_pt_objet = SeriesPT(serie[2]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array

            if 'POLYGON' in mask_objet.ecart_suv_max(nifti_array)  :
                print("PROBLEME SUV MAX POLYGON")
                erreur_polygone_max.append(serie)
            else : 

                result_mean = mask_objet.ecart_suv_mean(nifti_array)
                result_sd = mask_objet.ecart_SD(nifti_array)
                if (('POLYGON' in result_mean) and ('POLYGON' in result_sd)) : 
                    print("PROBLEME SUV MEAN ET SD POLYGONE")
                    print("ecart mean", result_mean)
                    print("ecart sd", result_sd)
                    erreur_polygone_mean_sd.append(serie)

                else : 
                    #si erreur sur les sd et mean QUE sur les ellipses, on tolère 
                    print("SMALL ERRORS ON ELLIPSE, MASK CORRECT")
                    #generation nifti PT
                    filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                    print("EXPORT NIFTI PT")
                    #generation nifti CT
                    serie_ct_objet = SeriesCT(serie[0])
                    serie_ct_objet.get_numpy_array()
                    filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                    serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                    print("EXPORT NIFTI CT")
                    #generation nifti mask
                    filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                    print("EXPORT NIFTI MASK")


    except Exception as err : 
        print(serie)
        print(err)

0
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
1
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
2
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
3
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
4
PROBLEME SUV MAX POLYGON
5
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
6
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
7
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
8
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
9
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
10
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
11
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT

In [5]:
print(len(erreur_polygone_max))
print(len(erreur_polygone_mean_sd))

89
3


In [6]:
#save as json the list of serie which have a mask problem 
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_serie_error_polygone_max', erreur_polygone_max)
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_serie_error_polygone_mean_sd', erreur_polygone_mean_sd)

In [ ]:
#CHECKER LE FLIP : AUCUNE SERIE GENERER AVEC LE FLIP 

In [19]:
#ON FLIP LE MASK ET ON REGARDE SI LES CALCLS SUV MAX MEAN SD SONT BONS
#SI OUI => GENERE NIFTI 

serie_with_flip = []
for serie in serie_error_mask: 
    print(serie_error_mask.index(serie))
    try : 
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array
            if (mask_objet.is_correct_suv(nifti_array, flip = True) == True) : #si mask correct APRES LE FLIP 
                print("MASK CORRECT APRES FLIP")
                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[2])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")
                #generation nifti mask
                mask_objet.flip_z(mask_4D)
                mask_4D_flip = mask_objet.mask_array
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D_flip)
                print("EXPORT NIFTI MASK")
                serie_with_flip.append(serie)

        else : 
            serie_pt_objet = SeriesPT(serie[2]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array
            if (mask_objet.is_correct_suv(nifti_array, flip = True) == True) : #si mask correct APRES LE FLIP 
                print("MASK CORRECT APRES FLIP")
                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[0])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")
                #generation nifti mask
                mask_objet.flip_z(mask_4D)
                mask_4D_flip = mask_objet.mask_array
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D_flip)
                print("EXPORT NIFTI MASK")
                serie_with_flip.append(serie)



    except Exception as err : 
        print(serie)
        print(err)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156


In [20]:
#APRES FLIP, REGARDER SI SMALL ERRORS SUR ELLIPSE
#SI PAS DE PROBLEMES SUR LES POLYGONES ON PEUT GENERER LE NIFTI 

In [4]:
#MASK AVEC FLIP FAUX : 
serie_problem = []

for serie in serie_error_mask: 
    print(serie_error_mask.index(serie))
    try : 
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array
            if 'POLYGON' in mask_objet.ecart_suv_max(nifti_array, flip = True) :
                print("PROBLEME SUV MAX POLYGON")
                serie_problem.append(serie)
            else : 
                result_mean = mask_objet.ecart_suv_mean(nifti_array, flip = True)
                result_sd = mask_objet.ecart_SD(nifti_array, flip = True)
                if (('POLYGON' not in result_mean) and ('POLYGON' not in result_sd)) : 
                    #si erreur sur les sd et mean QUE sur les ellipses, on tolère 
                    print("SMALL ERRORS ON ELLIPSE AFTER FLIP, MASK CORRECT")
                    #generation nifti PT
                    filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                    print("EXPORT NIFTI PT")
                    #generation nifti CT
                    serie_ct_objet = SeriesCT(serie[2])
                    serie_ct_objet.get_numpy_array()
                    filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                    serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                    print("EXPORT NIFTI CT")
                    #generation nifti mask
                    mask_objet.flip_z(mask_4D)
                    mask_4D_flip = mask_objet.mask_array
                    filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D_flip)
                    print("EXPORT NIFTI MASK")
                else : #sinon problème sur lles polygones
                    serie_problem.append(serie)





        else : 
            serie_pt_objet = SeriesPT(serie[2]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array
            if 'POLYGON' in mask_objet.ecart_suv_max(nifti_array, flip = True) :
                print("PROBLEME SUV MAX POLYGONE")
                serie_problem.append(serie)
            else : 
                result_mean = mask_objet.ecart_suv_mean(nifti_array, flip = True)
                result_sd = mask_objet.ecart_SD(nifti_array, flip = True)
                if (('POLYGON' not in result_mean) and ('POLYGON' not in result_sd)) : 
                    #si erreur sur les sd et mean QUE sur les ellipses, on tolère 
                    print("SMALL ERRORS ON ELLIPSE AFTER FLIP, MASK CORRECT")
                    #generation nifti PT
                    filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                    print("EXPORT NIFTI PT")
                    #generation nifti CT
                    serie_ct_objet = SeriesCT(serie[0])
                    serie_ct_objet.get_numpy_array()
                    filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                    serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                    print("EXPORT NIFTI CT")
                    #generation nifti mask
                    mask_objet.flip_z(mask_4D)
                    mask_4D_flip = mask_objet.mask_array
                    filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D_flip)
                    print("EXPORT NIFTI MASK")
                else : #sinon problème sur les polygones
                    serie_problem.append(serie)
            

    except Exception as err : 
        print(serie)
        print(err)

0
PROBLEME SUV MAX
1
PROBLEME SUV MAX
2
PROBLEME SUV MAX
3
PROBLEME SUV MAX
4
PROBLEME SUV MAX
5
PROBLEME SUV MAX
6
PROBLEME SUV MAX
7
PROBLEME SUV MAX
8
PROBLEME SUV MAX
9
PROBLEME SUV MAX
10
PROBLEME SUV MAX
11
PROBLEME SUV MAX
12
PROBLEME SUV MAX
13
PROBLEME SUV MAX
14
PROBLEME SUV MAX
15
PROBLEME SUV MAX
16
PROBLEME SUV MAX
17
PROBLEME SUV MAX
18
PROBLEME SUV MAX
19
PROBLEME SUV MAX
20
PROBLEME SUV MAX
21
PROBLEME SUV MAX
22
PROBLEME SUV MAX
23
PROBLEME SUV MAX
24
PROBLEME SUV MAX
25
PROBLEME SUV MAX
26
PROBLEME SUV MAX
27
PROBLEME SUV MAX
28
PROBLEME SUV MAX
29
PROBLEME SUV MAX
30
PROBLEME SUV MAX
31
PROBLEME SUV MAX
32
PROBLEME SUV MAX
33
PROBLEME SUV MAX
34
PROBLEME SUV MAX
35
PROBLEME SUV MAX
36
PROBLEME SUV MAX
37
PROBLEME SUV MAX
38
PROBLEME SUV MAX
39
PROBLEME SUV MAX
40
PROBLEME SUV MAX
41
PROBLEME SUV MAX
42
PROBLEME SUV MAX
43
PROBLEME SUV MAX
44
PROBLEME SUV MAX
45
PROBLEME SUV MAX
46
PROBLEME SUV MAX
47
PROBLEME SUV MAX
48
PROBLEME SUV MAX
49
PROBLEME SUV MAX
50
PROBLEM

In [3]:
#IMPORT JSON SERIES PHILIPS
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_untreated_philips_serie.json'
philips_serie = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        philips_serie.append(info)

print(len(philips_serie)) #203

203


In [15]:
#A FAIRE TOURNER
#SI PAS D'ERREUR SUR LES SUV MAX, ALORS OK 

philips_error = []
erreur_polygone_philips_max = []
erreur_polygone_philips_mean_sd = []
for serie in philips_serie : 
    print(philips_serie.index(serie))
    try : 
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array

            if 'POLYGON' in mask_objet.ecart_suv_max(nifti_array) :
                print("PROBLEME SUV MAX POLYGON")
                erreur_polygone_philips_max.append(serie)
            
            else : 

                result_mean = mask_objet.ecart_suv_mean(nifti_array)
                result_sd = mask_objet.ecart_SD(nifti_array)
                if (('POLYGON' in result_mean) and ('POLYGON' in result_sd)) : 
                    print("PROBLEME SUV MEAN ET SD POLYGONE")
                    print("ecart mean", result_mean)
                    print("ecart sd", result_sd)
                    erreur_polygone_philips_mean_sd.append(serie)

                else : 
                    #si erreur sur les sd et mean QUE sur les ellipses, on tolère 
                    print("SMALL ERRORS ON ELLIPSE, MASK CORRECT")
                    #generation nifti PT
                    filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                    print("EXPORT NIFTI PT")
                    #generation nifti CT
                    serie_ct_objet = SeriesCT(serie[2])
                    serie_ct_objet.get_numpy_array()
                    filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                    serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                    print("EXPORT NIFTI CT")
                    #generation nifti mask
                    filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                    print("EXPORT NIFTI MASK")


        else : 
            serie_pt_objet = SeriesPT(serie[2]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array

            if 'POLYGON' in mask_objet.ecart_suv_max(nifti_array)  :
                print("PROBLEME SUV MAX POLYGON")
                erreur_polygone_philips_max.append(serie)
            else : 

                result_mean = mask_objet.ecart_suv_mean(nifti_array)
                result_sd = mask_objet.ecart_SD(nifti_array)
                if (('POLYGON' in result_mean) and ('POLYGON' in result_sd)) : 
                    print("PROBLEME SUV MEAN ET SD POLYGONE")
                    print("ecart mean", result_mean)
                    print("ecart sd", result_sd)
                    erreur_polygone_philips_mean_sd.append(serie)

                else : 
                    #si erreur sur les sd et mean QUE sur les ellipses, on tolère 
                    print("SMALL ERRORS ON ELLIPSE, MASK CORRECT")
                    #generation nifti PT
                    filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                    print("EXPORT NIFTI PT")
                    #generation nifti CT
                    serie_ct_objet = SeriesCT(serie[0])
                    serie_ct_objet.get_numpy_array()
                    filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                    serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                    print("EXPORT NIFTI CT")
                    #generation nifti mask
                    filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                    print("EXPORT NIFTI MASK")


    except Exception as err : 
        print(serie)
        print(err)
        philips_error.append(serie)

ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
14
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
15
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
16
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
17
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
18
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
19
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
20
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
21
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
22
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
23
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
24
SMALL ERRORS ON

In [16]:
print(len(erreur_polygone_philips_max))
print(len(erreur_polygone_philips_mean_sd))
print(len(philips_error))
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_serie_error_philips_polygone_max', erreur_polygone_philips_max)
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_serie_philips_error', philips_error)


51
0
12


In [9]:
#COUNTS PHILIPS
counts = []
for philips in philips_serie : 
    if philips[1] == 'PT' : 
        serie_pt_objet = SeriesPT(philips[0])
        if serie_pt_objet.get_series_details()['series']['Units'] == 'CNTS' :
            counts.append(philips)

    else : 
        serie_pt_objet = SeriesPT(philips[2])
        if serie_pt_objet.get_series_details()['series']['Units'] == 'CNTS' :
            counts.append(philips)

In [25]:
print(len(counts))
counts[0]
print(counts[1])
serie_pt_objet = SeriesPT(counts[0][0])
#print(serie_pt_objet.get_series_details())

106
['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011102411002/1.2.250.1.59.0.8559.20120605125207.3263823/1.2.840.113704.7.32.0589.28.2.10.14.3165.4686.2.272.0.1338899965/1.2.840.113704.7.32.0589.28.2.10.14.3165.4686.2.272.0.1338899965', 'PT', '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011102411002/1.2.250.1.59.0.8559.20120605125207.3263823/1.2.840.113704.7.32.1.2.840.113704.1.111.5280.1338894543.14/1.2.840.113704.7.32.1.2.840.113704.1.111.5280.1338894543.14', 'CT', 'jem_ta_jun 5_2012.csv']


In [7]:
philips_serie[2]

['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101431010/1.2.124.113532.172.16.25.2.20120605.739.1104125/1.2.840.113704.1.111.5244.1339151088.11/imagys_AHL2011_13011101431010_f812bc05-4748-4b0a-bd1c-1e4a3954c530',
 'CT',
 '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101431010/1.2.124.113532.172.16.25.2.20120605.739.1104125/1.3.46.670589.28.2.12.21.6047.2953.2.2200.0.1339152332/imagys_AHL2011_13011101431010_37a1f66e-91db-4705-824e-33524671308e',
 'PT',
 'san_ma_jun 8_2012.csv']

In [21]:
json_liste = []
error_philips = []
for philips in philips_serie : 
    print(philips)
    subliste = []
    try : 
        if philips[1] == 'PT' : 
            serie_pt_objet = SeriesPT(philips[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            #study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, philips[4]), size_matrix)
            mask_4D = mask_objet.mask_array
            subliste.append(philips)
            subliste.append(mask_objet.calcul_suv(nifti_array))
            subliste.append(mask_objet.ecart_suv_max(nifti_array))
            print(mask_objet.calcul_suv(nifti_array))
            print(mask_objet.ecart_suv_max(nifti_array))
            json_liste.append(subliste)

        else : 
            serie_pt_objet = SeriesPT(philips[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            #study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, philips[4]), size_matrix)
            mask_4D = mask_objet.mask_array
            subliste.append(philips)
            subliste.append(mask_objet.calcul_suv(nifti_array))
            subliste.append(mask_objet.ecart_suv_max(nifti_array))
            print(mask_objet.calcul_suv(nifti_array))
            print(mask_objet.ecart_suv_max(nifti_array))
            json_liste.append(subliste)

    except Exception as err : 
        print(serie)
        print(err)
        error_philips.append(philips)


print(len(json_liste))

 2.21, 'SD': 0.49}, 6: {'SUV_max': 4.46, 'SUV_mean': 2.57, 'SD': 0.66}}
[]
['/media/salim/DD 2To/AHL2011_Validated_DICOMS/53011101851003/1.2.250.1.38.2.1.12.1617369990219.20120110135439.1941172235/1.3.46.670589.28.2.12.30.26377.35752.2.3036.0.1326300008/1.3.46.670589.28.2.12.30.26377.35752.2.3036.0.1326300008', 'PT', '/media/salim/DD 2To/AHL2011_Validated_DICOMS/53011101851003/1.2.250.1.38.2.1.12.1617369990219.20120110135439.1941172235/1.2.840.113704.1.111.4680.1326211698.8/1.2.840.113704.1.111.4680.1326211698.8', 'CT', 'fra_ni_jan 10_2012.csv']
20120110152044
{1: {'SUV_max': 7.77, 'SUV_mean': 4.37, 'SD': 0.97}, 2: {'SUV_max': 12.92, 'SUV_mean': 6.79, 'SD': 1.46}, 3: {'SUV_max': 16.55, 'SUV_mean': 9.26, 'SD': 2.08}, 4: {'SUV_max': 11.88, 'SUV_mean': 6.61, 'SD': 1.46}, 5: {'SUV_max': 15.35, 'SUV_mean': 8.28, 'SD': 1.85}, 6: {'SUV_max': 6.08, 'SUV_mean': 3.25, 'SD': 0.7}, 7: {'SUV_max': 15.35, 'SUV_mean': 9.02, 'SD': 2.09}, 8: {'SUV_max': 6.46, 'SUV_mean': 3.65, 'SD': 0.81}, 9: {'SUV_max

ZeroDivisionError: float division by zero

In [ ]:
['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101431017/1.2.124.113532.172.16.25.2.20130717.529.1327224/1.2.840.113704.1.111.7840.1374146611.11/1.2.840.113704.1.111.7840.1374146611.11', 'CT', '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101431017/1.2.124.113532.172.16.25.2.20130717.529.1327224/1.3.46.670589.28.2.12.30.26421.36112.2.2640.0.1374191278/1.3.46.670589.28.2.12.30.26421.36112.2.2640.0.1374191278', 'PT', 'ber_gr_jul 18_2013.csv']
20130718121208
{1: {'SUV_max': 8.69, 'SUV_mean': 5.45, 'SD': 1.45}, 2: {'SUV_max': 3.36, 'SUV_mean': 2.01, 'SD': 0.64}, 3: {'SUV_max': 2.38, 'SUV_mean': 1.51, 'SD': 0.34}, 4: {'SUV_max': 4.18, 'SUV_mean': 2.28, 'SD': 0.49}, 5: {'SUV_max': 5.63, 'SUV_mean': 3.39, 'SD': 0.81}, 6: {'SUV_max': 4.45, 'SUV_mean': 2.73, 'SD': 0.74}, 7: {'SUV_max': 3.15, 'SUV_mean': 1.82, 'SD': 0.41}, 8: {'SUV_max': 5.21, 'SUV_mean': 2.93, 'SD': 0.6}, 9: {'SUV_max': 5.21, 'SUV_mean': 2.85, 'SD': 0.55}, 10: {'SUV_max': 3.3, 'SUV_mean': 1.84, 'SD': 0.44}, 11: {'SUV_max': 2.7, 'SUV_mean': 1.41, 'SD': 0.28}, 12: {'SUV_max': 2.14, 'SUV_mean': 1.24, 'SD': 0.35}, 13: {'SUV_max': 8.33, 'SUV_mean': 5.73, 'SD': 0.95}, 14: {'SUV_max': 7.51, 'SUV_mean': 5.26, 'SD': 1.05}, 15: {'SUV_max': 2.73, 'SUV_mean': 1.64, 'SD': 0.4}, 16: {'SUV_max': 2.48, 'SUV_mean': 1.56, 'SD': 0.34}, 17: {'SUV_max': 2.46, 'SUV_mean': 1.7, 'SD': 0.28}, 18: {'SUV_max': 2.65, 'SUV_mean': 1.93, 'SD': 0.36}}
[1, 'POLYGON', 1.0700000000000003, 2, 'POLYGON', 0.41000000000000014, 3, 'POLYGON', 0.29000000000000004, 4, 'POLYGON', 0.5100000000000007, 5, 'POLYGON', 0.7000000000000002, 6, 'POLYGON', 0.5499999999999998, 7, 'POLYGON', 0.3900000000000001, 8, 'POLYGON', 0.6399999999999997, 9, 'POLYGON', 0.6399999999999997, 10, 'POLYGON', 0.40000000000000036, 11, 'POLYGON', 0.33999999999999986, 13, 'POLYGON', 1.0199999999999996, 14, 'POLYGON', 0.9199999999999999, 15, 'POLYGON', 0.33000000000000007, 17, 'POLYGON', 0.31000000000000005, 18, 'POLYGON', 0.33000000000000007]




In [ ]:
['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101431011/1.2.124.113532.172.16.25.2.20120716.81922.1126423/1.2.840.113704.1.111.7108.1342703962.11/imagys_AHL2011_13011101431011_4f47b5d8-c16e-4f32-842c-c9c3d36b7124', 'CT', '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101431011/1.2.124.113532.172.16.25.2.20120716.81922.1126423/1.3.46.670589.28.2.12.21.6046.64205.2.1680.0.1342705432/1.3.46.670589.28.2.12.21.6046.64205.2.1680.0.1342705432', 'PT', 'aul_fl_jul 19_2012.csv']
20120719141344
{1: {'SUV_max': 8.77, 'SUV_mean': 5.34, 'SD': 1.23}, 2: {'SUV_max': 11.7, 'SUV_mean': 6.67, 'SD': 1.55}, 3: {'SUV_max': 8.83, 'SUV_mean': 5.13, 'SD': 1.25}, 4: {'SUV_max': 11.02, 'SUV_mean': 6.43, 'SD': 1.49}, 5: {'SUV_max': 15.04, 'SUV_mean': 8.95, 'SD': 2.27}, 6: {'SUV_max': 13.89, 'SUV_mean': 8.07, 'SD': 2.3}, 7: {'SUV_max': 12.08, 'SUV_mean': 7.26, 'SD': 1.88}, 8: {'SUV_max': 7.35, 'SUV_mean': 4.37, 'SD': 1.09}, 9: {'SUV_max': 6.29, 'SUV_mean': 3.86, 'SD': 0.99}, 10: {'SUV_max': 8.32, 'SUV_mean': 5.19, 'SD': 1.39}, 11: {'SUV_max': 11.72, 'SUV_mean': 6.58, 'SD': 1.46}, 12: {'SUV_max': 13.35, 'SUV_mean': 7.38, 'SD': 1.69}, 13: {'SUV_max': 12.01, 'SUV_mean': 6.71, 'SD': 1.46}, 14: {'SUV_max': 7.63, 'SUV_mean': 4.66, 'SD': 1.2}, 15: {'SUV_max': 6.85, 'SUV_mean': 4.02, 'SD': 0.99}, 16: {'SUV_max': 7.74, 'SUV_mean': 4.19, 'SD': 0.97}, 17: {'SUV_max': 7.97, 'SUV_mean': 4.51, 'SD': 1.02}, 18: {'SUV_max': 5.06, 'SUV_mean': 3.03, 'SD': 0.8}, 19: {'SUV_max': 6.58, 'SUV_mean': 4.04, 'SD': 1.05}, 20: {'SUV_max': 3.91, 'SUV_mean': 2.57, 'SD': 0.63}, 21: {'SUV_max': 16.96, 'SUV_mean': 9.12, 'SD': 1.67}, 22: {'SUV_max': 12.9, 'SUV_mean': 6.81, 'SD': 1.35}, 23: {'SUV_max': 4.7, 'SUV_mean': 2.82, 'SD': 0.81}, 24: {'SUV_max': 7.77, 'SUV_mean': 4.95, 'SD': 1.38}, 25: {'SUV_max': 11.02, 'SUV_mean': 6.11, 'SD': 1.27}, 26: {'SUV_max': 3.68, 'SUV_mean': 2.17, 'SD': 0.56}, 27: {'SUV_max': 6.23, 'SUV_mean': 3.7, 'SD': 0.91}, 28: {'SUV_max': 2.93, 'SUV_mean': 2.23, 'SD': 0.34}, 29: {'SUV_max': 4.93, 'SUV_mean': 3.56, 'SD': 0.66}, 30: {'SUV_max': 5.02, 'SUV_mean': 3.42, 'SD': 0.68}, 31: {'SUV_max': 5.01, 'SUV_mean': 4.18, 'SD': 0.42}, 32: {'SUV_max': 4.87, 'SUV_mean': 3.56, 'SD': 0.63}, 33: {'SUV_max': 4.77, 'SUV_mean': 3.33, 'SD': 0.69}, 34: {'SUV_max': 4.95, 'SUV_mean': 3.52, 'SD': 0.59}, 35: {'SUV_max': 4.69, 'SUV_mean': 3.74, 'SD': 0.5}, 36: {'SUV_max': 4.54, 'SUV_mean': 3.59, 'SD': 0.66}, 37: {'SUV_max': 4.81, 'SUV_mean': 4.04, 'SD': 0.36}, 38: {'SUV_max': 4.93, 'SUV_mean': 3.12, 'SD': 0.72}, 39: {'SUV_max': 5.12, 'SUV_mean': 3.13, 'SD': 0.68}, 40: {'SUV_max': 4.72, 'SUV_mean': 3.33, 'SD': 0.69}}
[1, 'POLYGON', 0.7000000000000011, 2, 'POLYGON', 0.9400000000000013, 3, 'POLYGON', 0.6999999999999993, 4, 'POLYGON', 0.8800000000000008, 5, 'POLYGON', 1.1999999999999993, 6, 'POLYGON', 1.1099999999999994, 7, 'POLYGON', 0.9599999999999991, 8, 'POLYGON', 0.5900000000000007, 9, 'POLYGON', 0.5099999999999998, 10, 'POLYGON', 0.6699999999999999, 11, 'POLYGON', 0.9299999999999997, 12, 'POLYGON', 1.0600000000000005, 13, 'POLYGON', 0.41999999999999993, 14, 'POLYGON', 0.6100000000000003, 15, 'POLYGON', 0.5500000000000007, 16, 'POLYGON', 0.6099999999999994, 17, 'POLYGON', 0.6399999999999997, 18, 'POLYGON', 0.40000000000000036, 19, 'POLYGON', 0.5199999999999996, 20, 'POLYGON', 0.3099999999999996, 21, 'POLYGON', 1.3499999999999979, 22, 'POLYGON', 1.0299999999999994, 23, 'POLYGON', 0.3799999999999999, 24, 'POLYGON', 0.620000000000001, 25, 'POLYGON', 0.8800000000000008, 26, 'POLYGON', 0.29000000000000004, 27, 'POLYGON', 0.4899999999999993, 28, 'POLYGON', 0.22999999999999998, 29, 'POLYGON', 0.39000000000000057, 30, 'POLYGON', 0.40000000000000036, 31, 'POLYGON', 0.40000000000000036, 32, 'POLYGON', 0.3899999999999997, 33, 'POLYGON', 0.3800000000000008, 34, 'POLYGON', 0.3899999999999997, 35, 'POLYGON', 0.3699999999999992, 36, 'POLYGON', 0.3600000000000003, 37, 'POLYGON', 0.3800000000000008, 38, 'POLYGON', 0.40000000000000036, 39, 'POLYGON', 0.41000000000000014, 40, 'POLYGON', 0.3700000000000001]

In [7]:
serie_pt_objet = SeriesPT('/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101431011/1.2.124.113532.172.16.25.2.20120716.81922.1126423/1.3.46.670589.28.2.12.21.6046.64205.2.1680.0.1342705432/1.3.46.670589.28.2.12.21.6046.64205.2.1680.0.1342705432')
print(serie_pt_objet.get_series_details())
serie_pt_objet.get_numpy_array()

{'series': {'Manufacturer': 'Philips Medical Systems', 'Modality': 'PT', 'SeriesDate': '20120719', 'SeriesTime': '152145', 'SeriesDescription': '[WB_CTAC] Body', 'SeriesInstanceUID': '1.3.46.670589.28.2.12.21.6046.64205.2.1680.0.1342705432', 'SeriesNumber': "54320", 'AcquisitionDate': '20120719', 'AcquisitionTime': '153105.007', 'NumberOfSlices': 213, 'ImageInAcquisition': 'Undefined', 'Units': 'BQML', 'DecayCorrection': 'START', 'ImageType': ['ORIGINAL', 'PRIMARY'], 'PixelSpacing': ["4.0", "4.0"]}, 'study': {'AccessionNumber': 'PET0', 'InstitutionName': '', 'StudyDate': '20120719', 'StudyTime': '152142', 'StudyDescription': 'PET0', 'StudyID': '15659', 'StudyInstanceUID': '1.2.124.113532.172.16.25.2.20120716.81922.1126423', 'PatientWeight': "63.0", 'PatientHeight': 'Undefined'}, 'patient': {'PatientName': 'AUL^FL', 'PatientID': '13011101431011', 'PatientBirthDate': '19000101', 'PatientSex': 'F'}, 'path': '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101431011/1.2.124.113532.172.16

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [9]:
serie_pt_objet = SeriesPT('/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101431011/1.2.124.113532.172.16.25.2.20120716.81922.1126423/1.3.46.670589.28.2.12.21.6046.64205.2.1680.0.1342705432/1.3.46.670589.28.2.12.21.6046.64205.2.1680.0.1342705432')
print(serie_pt_objet.get_series_details())
nifti_array = serie_pt_objet.get_numpy_array()
size_matrix = serie_pt_objet.get_size_matrix()
mask_objet = MaskBuilder(os.path.join(csv_directory, 'aul_fl_jul 19_2012.csv'), size_matrix)
mask_4D = mask_objet.mask_array

mask_objet.calcul_suv(nifti_array)

mask_objet.ecart_suv_max(nifti_array)

{'series': {'Manufacturer': 'Philips Medical Systems', 'Modality': 'PT', 'SeriesDate': '20120719', 'SeriesTime': '152145', 'SeriesDescription': '[WB_CTAC] Body', 'SeriesInstanceUID': '1.3.46.670589.28.2.12.21.6046.64205.2.1680.0.1342705432', 'SeriesNumber': "54320", 'AcquisitionDate': '20120719', 'AcquisitionTime': '153105.007', 'NumberOfSlices': 213, 'ImageInAcquisition': 'Undefined', 'Units': 'BQML', 'DecayCorrection': 'START', 'ImageType': ['ORIGINAL', 'PRIMARY'], 'PixelSpacing': ["4.0", "4.0"]}, 'study': {'AccessionNumber': 'PET0', 'InstitutionName': '', 'StudyDate': '20120719', 'StudyTime': '152142', 'StudyDescription': 'PET0', 'StudyID': '15659', 'StudyInstanceUID': '1.2.124.113532.172.16.25.2.20120716.81922.1126423', 'PatientWeight': "63.0", 'PatientHeight': 'Undefined'}, 'patient': {'PatientName': 'AUL^FL', 'PatientID': '13011101431011', 'PatientBirthDate': '19000101', 'PatientSex': 'F'}, 'path': '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101431011/1.2.124.113532.172.16

[1,
 'POLYGON',
 0.7000000000000011,
 2,
 'POLYGON',
 0.9400000000000013,
 3,
 'POLYGON',
 0.6999999999999993,
 4,
 'POLYGON',
 0.8800000000000008,
 5,
 'POLYGON',
 1.1999999999999993,
 6,
 'POLYGON',
 1.1099999999999994,
 7,
 'POLYGON',
 0.9599999999999991,
 8,
 'POLYGON',
 0.5900000000000007,
 9,
 'POLYGON',
 0.5099999999999998,
 10,
 'POLYGON',
 0.6699999999999999,
 11,
 'POLYGON',
 0.9299999999999997,
 12,
 'POLYGON',
 1.0600000000000005,
 13,
 'POLYGON',
 0.41999999999999993,
 14,
 'POLYGON',
 0.6100000000000003,
 15,
 'POLYGON',
 0.5500000000000007,
 16,
 'POLYGON',
 0.6099999999999994,
 17,
 'POLYGON',
 0.6399999999999997,
 18,
 'POLYGON',
 0.40000000000000036,
 19,
 'POLYGON',
 0.5199999999999996,
 20,
 'POLYGON',
 0.3099999999999996,
 21,
 'POLYGON',
 1.3499999999999979,
 22,
 'POLYGON',
 1.0299999999999994,
 23,
 'POLYGON',
 0.3799999999999999,
 24,
 'POLYGON',
 0.620000000000001,
 25,
 'POLYGON',
 0.8800000000000008,
 26,
 'POLYGON',
 0.29000000000000004,
 27,
 'POLYGON',
 0

In [ ]:
#regarder les erreurs de max autres que philips 
#regarder les erreurs philips 
#problème de datetime dans certaines séries : 
        #Error generating result array time data '' does not match format '%Y%m%d%H%M%S'

In [ ]:
#philips -> MIP MASK 